In [1]:
import pandas as pd
import pandas_datareader as web

In [13]:
# load dataset
#scraping wikipedia to fetch S&P 500 stock list
snp500url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
SP_data = pd.read_html(snp500url)

dji = (
    pd.read_html('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average')[1]
)
dji_symbols = dji.Symbol.tolist()